In [1]:
import torch

print(torch.__version__)

2.2.1


In [2]:
print(torch.backends.mps.is_available())

print(torch.backends.mps.is_built())

True
True


In [3]:
from roboflow import Roboflow

In [4]:
rf = Roboflow(api_key="YXYlRGsiGi26trClZiMv")

In [5]:
shelf_project = rf.workspace("shelfdetect-yzkro").project("shelves-ugxt3")
shelf_dataset = shelf_project.version(3).download("yolov8", location="./datasets/shelves", overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/shelves in yolov8:: 100%|██████████| 2472/2472 [00:00<00:00, 3379.88it/s]


In [6]:
grocery_project = rf.workspace("grocery-barcodes").project("sku110k-1000-2000")
grocery_dataset = grocery_project.version(3).download("yolov8", location="./datasets/groceries", overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/groceries in yolov8:: 100%|██████████| 5562/5562 [00:01<00:00, 3596.76it/s]


In [7]:
logo_project = rf.workspace("philip-hawkins-ocdim").project("openlogo-zgtgi")
logo_dataset = logo_project.version(4).download("yolov8", location="./datasets/brand-logo", overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/brand-logo in yolov8:: 100%|██████████| 19246/19246 [00:04<00:00, 4035.20it/s]


In [8]:
import os
import shutil

In [9]:
os.getcwd()

'/Users/priyanshutuli/Desktop/Planogram_Optimization'

In [10]:
logo_dataset_path = "./datasets/brand-logo"
groceries_dataset_path = "./datasets/groceries"
shelves_dataset_path = "./datasets/shelves"

In [11]:
for split in ["train", "test", "valid"]:
    os.makedirs(f"./datasets/complete_dataset/{split}/images", exist_ok=True)
    os.makedirs(f"./datasets/complete_dataset/{split}/labels", exist_ok=True)

In [12]:
for dataset_path in [groceries_dataset_path, shelves_dataset_path]:
    for root, dirs, files in os.walk(dataset_path):
        if "train/images" in root:
            for file in files:
                file_path = os.path.join(root, file)
                shutil.copy(file_path, "./datasets/complete_dataset/train/images")
        elif "train/labels" in root:
            for file in files:
                file_path = os.path.join(root, file)
                if dataset_path in [shelves_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "0", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                elif dataset_path in [groceries_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "1", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                shutil.copy(file_path, "./datasets/complete_dataset/train/labels")
        elif "test/images" in root:
            for file in files:
                file_path = os.path.join(root, file)
                shutil.copy(file_path, "./datasets/complete_dataset/test/images")
        elif "test/labels" in root:
            for file in files:
                file_path = os.path.join(root, file)
                if dataset_path in [shelves_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "0", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                elif dataset_path in [groceries_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "1", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                shutil.copy(file_path, "./datasets/complete_dataset/test/labels")
        elif "valid/images" in root:
            for file in files:
                file_path = os.path.join(root, file)
                shutil.copy(file_path, "./datasets/complete_dataset/valid/images")
        elif "valid/labels" in root:
            for file in files:
                file_path = os.path.join(root, file)
                if dataset_path in [shelves_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "0", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                elif dataset_path in [groceries_dataset_path]:
                    with open(file_path, "r+") as fp:
                        labels = fp.readlines()
                        for i, label in enumerate(labels):
                            labels[i] = label.replace("0", "1", 1)
                        fp.seek(0)
                        fp.writelines(labels)
                shutil.copy(file_path, "./datasets/complete_dataset/valid/labels")
        

In [13]:
from ultralytics import YOLO

In [14]:
model_xlarge = YOLO('yolov8x.pt', task="detect")

In [15]:
model_xlarge.names.values()

dict_values(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'])

In [16]:
len(model_xlarge.names)

80

In [17]:
import comet_ml

comet_ml.init(api_key="MECzH3F5WGRgda13HDbKcVcMt")

COMET INFO: Valid Comet API Key saved in /Users/priyanshutuli/.comet.config (set COMET_CONFIG to change where it is saved).


In [18]:
source = "./inference_images"
results = model_xlarge.predict(source, conf=0.5, imgsz=640, iou=0.7, save=True, device="mps", seed=0)


image 1/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0205BB3A-F171-4F92-A552-46B64F835D30_IMG_3467.jpeg: 640x480 1 surfboard, 370.1ms
image 2/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0F488364-219C-4A21-94B2-0B6A884420DD_IMG_0041.jpeg: 640x480 (no detections), 114.5ms
image 3/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/12B22766-4862-470A-BC05-DAB41EFB537D_IMG_0986.jpeg: 640x480 1 person, 124.6ms
image 4/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/177E5396-E9E2-47EC-B481-AA787E402CC7_IMG_1759.jpeg: 640x480 1 refrigerator, 112.8ms
image 5/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/1AE868B4-02F6-4F6D-B09C-3B65042035D7_IMG_3525.jpeg: 640x480 (no detections), 106.8ms
image 6/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/20240207_205553-COLLAGE.jpg: 640x640 1 person, 189.7ms
image 7/933 /Users/priyanshutuli/Desktop/Plan

In [19]:
model_nano = YOLO("yolov8n.pt", task="detect")

In [20]:
torch.mps.empty_cache()
results = model_nano.train(data='/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/complete_dataset/data.yaml', epochs=20, imgsz=640, seed=0, device='cpu', save=True, patience=10, batch=16, verbose=True,
                      plots=True, cos_lr=True, lr0=1e-3, lrf=1e-6, optimizer='AdamW', deterministic=True, save_period = 5, amp=True, val=True)

New https://pypi.org/project/ultralytics/8.1.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.11 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/complete_dataset/data.yaml, epochs=20, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=N

In [22]:
results_finetuning = model_nano.predict(source, conf=0.5, imgsz=640, iou=0.4, save=True, device="mps", seed=0)


image 1/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0205BB3A-F171-4F92-A552-46B64F835D30_IMG_3467.jpeg: 640x480 (no detections), 411.9ms
image 2/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0F488364-219C-4A21-94B2-0B6A884420DD_IMG_0041.jpeg: 640x480 (no detections), 26.1ms
image 3/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/12B22766-4862-470A-BC05-DAB41EFB537D_IMG_0986.jpeg: 640x480 1 object, 27.1ms
image 4/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/177E5396-E9E2-47EC-B481-AA787E402CC7_IMG_1759.jpeg: 640x480 18 objects, 24.1ms
image 5/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/1AE868B4-02F6-4F6D-B09C-3B65042035D7_IMG_3525.jpeg: 640x480 73 objects, 26.6ms
image 6/933 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/20240207_205553-COLLAGE.jpg: 640x640 29 objects, 37.9ms
image 7/933 /Users/priyanshutuli/Desktop/Planogram_Op

In [28]:
for k,v in model_nano.named_parameters():
    print(k)

model.model.0.conv.weight
model.model.0.bn.weight
model.model.0.bn.bias
model.model.1.conv.weight
model.model.1.bn.weight
model.model.1.bn.bias
model.model.2.cv1.conv.weight
model.model.2.cv1.bn.weight
model.model.2.cv1.bn.bias
model.model.2.cv2.conv.weight
model.model.2.cv2.bn.weight
model.model.2.cv2.bn.bias
model.model.2.m.0.cv1.conv.weight
model.model.2.m.0.cv1.bn.weight
model.model.2.m.0.cv1.bn.bias
model.model.2.m.0.cv2.conv.weight
model.model.2.m.0.cv2.bn.weight
model.model.2.m.0.cv2.bn.bias
model.model.3.conv.weight
model.model.3.bn.weight
model.model.3.bn.bias
model.model.4.cv1.conv.weight
model.model.4.cv1.bn.weight
model.model.4.cv1.bn.bias
model.model.4.cv2.conv.weight
model.model.4.cv2.bn.weight
model.model.4.cv2.bn.bias
model.model.4.m.0.cv1.conv.weight
model.model.4.m.0.cv1.bn.weight
model.model.4.m.0.cv1.bn.bias
model.model.4.m.0.cv2.conv.weight
model.model.4.m.0.cv2.bn.weight
model.model.4.m.0.cv2.bn.bias
model.model.4.m.1.cv1.conv.weight
model.model.4.m.1.cv1.bn.weigh

In [29]:
from ultralytics.engine import trainer

In [30]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

In [33]:
model_nano_froozen_layers = YOLO(model="yolov8n.pt", task="detect")
model_nano_froozen_layers.add_callback("on_train_start", freeze_layer)

In [36]:
for i, (name, param) in enumerate(model_xlarge.named_parameters()):
    print(i, name)

0 model.model.0.conv.weight
1 model.model.0.conv.bias
2 model.model.1.conv.weight
3 model.model.1.conv.bias
4 model.model.2.cv1.conv.weight
5 model.model.2.cv1.conv.bias
6 model.model.2.cv2.conv.weight
7 model.model.2.cv2.conv.bias
8 model.model.2.m.0.cv1.conv.weight
9 model.model.2.m.0.cv1.conv.bias
10 model.model.2.m.0.cv2.conv.weight
11 model.model.2.m.0.cv2.conv.bias
12 model.model.2.m.1.cv1.conv.weight
13 model.model.2.m.1.cv1.conv.bias
14 model.model.2.m.1.cv2.conv.weight
15 model.model.2.m.1.cv2.conv.bias
16 model.model.2.m.2.cv1.conv.weight
17 model.model.2.m.2.cv1.conv.bias
18 model.model.2.m.2.cv2.conv.weight
19 model.model.2.m.2.cv2.conv.bias
20 model.model.3.conv.weight
21 model.model.3.conv.bias
22 model.model.4.cv1.conv.weight
23 model.model.4.cv1.conv.bias
24 model.model.4.cv2.conv.weight
25 model.model.4.cv2.conv.bias
26 model.model.4.m.0.cv1.conv.weight
27 model.model.4.m.0.cv1.conv.bias
28 model.model.4.m.0.cv2.conv.weight
29 model.model.4.m.0.cv2.conv.bias
30 model.m

In [37]:
torch.mps.empty_cache()
results_froozen_layers = model_nano_froozen_layers.train(data='/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/complete_dataset/data.yaml', epochs=50, imgsz=640, seed=0, device='cpu', save=True, patience=10, batch=16, verbose=True,
                      plots=True, cos_lr=True, lr0=1e-3, lrf=1e-6, optimizer='AdamW', deterministic=True, save_period = 5, amp=True, val=True)

New https://pypi.org/project/ultralytics/8.1.19 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.11 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/complete_dataset/data.yaml, epochs=50, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=N

Freezing 10 layers
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.cv1.conv.weight
freezing model.2.cv1.bn.weight
freezing model.2.cv1.bn.bias
freezing model.2.cv2.conv.weight
freezing model.2.cv2.bn.weight
freezing model.2.cv2.bn.bias
freezing model.2.m.0.cv1.conv.weight
freezing model.2.m.0.cv1.bn.weight
freezing model.2.m.0.cv1.bn.bias
freezing model.2.m.0.cv2.conv.weight
freezing model.2.m.0.cv2.bn.weight
freezing model.2.m.0.cv2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.cv1.conv.weight
freezing model.4.cv1.bn.weight
freezing model.4.cv1.bn.bias
freezing model.4.cv2.conv.weight
freezing model.4.cv2.bn.weight
freezing model.4.cv2.bn.bias
freezing model.4.m.0.cv1.conv.weight
freezing model.4.m.0.cv1.bn.weight
freezing model.4.m.0.cv1.bn.bias
freezing model.4.m.0.cv2.conv.weight
freezing model

       1/50         0G      1.864      1.304      1.209       1792        640: 100%|██████████| 225/225 [28:29<00:00,  7.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:35<00:00,  8.80s/it]
                   all        126      10747      0.316      0.385      0.297      0.153

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50         0G      1.736      1.117      1.146       1663        640: 100%|██████████| 225/225 [27:49<00:00,  7.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:33<00:00,  8.44s/it]
                   all        126      10747      0.276      0.288      0.274      0.153

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/50         0G      1.678      1.051      1.123       1246        640: 100%|██████████| 225/225 [26:56<00:00,  7.19s/it]
     